In [1]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
from solar_power.environment_prediction_model import SolarPredictionModelExtended 
from solar_power.solar_prediction_model import FeatureSolarPredictionModel
from electrical_load.load_prediction_model import BuildingEnergyModel

# * 모델 선언
# * 랜덤 시드 값을 모델 간 동일하게 42로 설정
# * 테스트 데이터 개수는 모델 간 동일하게 12 설정, 12/62 = 19.35%

random_state = 42
test_size = 12 

# * 기상 데이터 to 환경 감시 데이터 예측 모델
env_pred_model = SolarPredictionModelExtended("./solar_power/merged_result.csv", random_state=random_state, test_size=test_size)
env_pred_model.load_data()
env_pred_model.hyperparameter_tuning()
env_pred_model.train_model()

# * 환경 감시 데이터 to 태양광 발전량 예측 모델
solar_pred_model = FeatureSolarPredictionModel("./solar_power/merged_result.csv", random_state=random_state, test_size=test_size)
solar_pred_model.load_data()
solar_pred_model.feature_selection()
solar_pred_model.train_model()

# * 건물 전기 부하 예측 모델
energy_pred_model = BuildingEnergyModel("./electrical_load/merged_data.csv", random_state=random_state, test_size=test_size)
energy_pred_model.load_data()
energy_pred_model.train_model()

In [8]:

# * 태양광 발전량 예측 모델에 입력할 정규화된 입력 데이터
# * test_x 는 1시간 단위의 데이터
# * test_date 는 예측할 시간의 날짜

import numpy as np
rand_int = np.random.randint(0, len(env_pred_model.X_test))

test_x = env_pred_model.X_test.iloc[rand_int:rand_int+1, :]
test_index = test_x.index[0]
test_date = env_pred_model.data.loc[test_index, 'day']

test_x


,강수확률,일최저기온,하늘상태,일최고기온,습도,풍향,풍속,time
137,0.2,1.0,0.666667,0.571429,0.333333,0.620961,0.532258,0.869565


In [9]:

# * 1시간 단위의 데이터와 같은 날짜의 데이터 24개를 합쳐 same_day_X_test 생성

same_day_data = env_pred_model.data[env_pred_model.data['day'] == test_date]
same_day_data_indexes = same_day_data.index
same_day_X_test = env_pred_model.X_test.loc[env_pred_model.X_test.index.isin(same_day_data_indexes)]

same_day_X_test

,강수확률,일최저기온,하늘상태,일최고기온,습도,풍향,풍속,time
120,0.2,1.0,0.666667,0.571429,0.833333,0.500000,0.096774,0.500000
121,0.2,1.0,0.666667,0.571429,0.916667,0.552264,0.129032,0.521739
122,0.3,1.0,1.000000,0.571429,0.916667,0.456422,0.177419,0.543478
123,0.3,1.0,1.000000,0.571429,0.916667,0.500000,0.161290,0.565217
124,0.3,1.0,1.000000,0.571429,0.916667,0.687303,0.161290,0.586957
125,0.2,1.0,0.666667,0.571429,0.916667,0.779596,0.161290,0.608696
126,0.3,1.0,1.000000,0.571429,0.916667,0.786788,0.177419,0.630435
127,0.3,1.0,1.000000,0.571429,0.833333,0.679184,0.258065,0.652174
128,0.3,1.0,1.000000,0.571429,0.750000,0.711309,0.322581,0.673913
129,0.2,1.0,0.666667,0.571429,0.666667,0.679184,0.338710,0.695652


# 태양광 발전량 예측

In [10]:

# * 예측한 태양광 발전량 데이터

res_solar = env_pred_model.predict(same_day_X_test) # 기상 데이터 -> 환경감시 데이터
res_solar = solar_pred_model.predict(res_solar) # 환경감시 데이터 -> 태양광 발전량 데이터
res_solar

array([  6.47454955,   7.47409053,   3.96709808,   3.96709808,
         3.96709808,   6.89904253,   3.96709808,  45.85679013,
       202.20285325, 525.7670981 , 656.31095003, 669.65130045,
       648.66346745, 669.09246129, 720.27766471, 693.87096247,
       577.12866052, 422.39476555, 210.30452074,  27.72514495,
        -3.28442807,  -3.61901513,  -3.39602418,  -3.39602418])

In [11]:
# * 실제 태양광 발전량

same_day_y_test = solar_pred_model.y_test.loc[solar_pred_model.y_test.index.isin(same_day_data_indexes)]
same_day_y_test

,축구장,학생회관,중앙창고,학사과정,다산빌딩,시설관리동,대학C동,동물실험동,중앙도서관,LG도서관,신재생에너지동,삼성환경동,중앙연구기기센터,산업협력관,기숙사 B동
120,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
122,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
123,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
124,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
126,1.0,1.1,0.6,0.5,0.2,1.4,0.5,1.1,0.7,0.9,1.1,0.0,0.9,3.0,1.2
127,2.8,2.0,1.3,0.9,0.4,2.9,0.7,1.7,1.1,1.3,3.2,2.2,4.0,4.8,1.9
128,19.6,9.8,10.3,9.5,7.5,25.5,3.9,9.1,7.9,9.9,20.5,15.4,29.9,23.4,14.0
129,22.7,9.0,6.8,11.3,7.2,21.3,5.3,7.9,7.3,14.2,16.3,12.2,24.4,23.6,19.4


# 전기 부하량 예측

In [12]:
# * 부하량 예측 모델에 입력할 정규화된 입력 데이터

same_day_X_test = energy_pred_model.X_test.loc[energy_pred_model.X_test.index.isin(same_day_data_indexes)]
same_day_X_test

,time,1시간기온,1시간강수량,일최고기온,일최저기온,weekday_Friday,weekday_Monday,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday
120,0,26.0,0.0,31.0,26.0,False,False,False,False,False,False,True
121,1,26.0,0.0,31.0,26.0,False,False,False,False,False,False,True
122,2,25.0,0.0,31.0,26.0,False,False,False,False,False,False,True
123,3,25.0,0.0,31.0,26.0,False,False,False,False,False,False,True
124,4,25.0,0.0,31.0,26.0,False,False,False,False,False,False,True
125,5,25.0,0.0,31.0,26.0,False,False,False,False,False,False,True
126,6,25.0,0.0,31.0,26.0,False,False,False,False,False,False,True
127,7,26.0,0.0,31.0,26.0,False,False,False,False,False,False,True
128,8,27.0,0.0,31.0,26.0,False,False,False,False,False,False,True
129,9,29.0,0.0,31.0,26.0,False,False,False,False,False,False,True


In [13]:
# * 예측한 전기 부하량

res_load = energy_pred_model.predict(same_day_X_test)
res_load

,중앙P/P_석사,전기전자컴퓨터공학동_석사,신소재공학_석사,생명과학동_석사,기계공학동_석사,LG도서관_석사,창업B동_석사,금호관_석사,"냉각수순환펌프1,2,3_석사","냉각수순환펌프4,5,6_석사",...,대학B동(E)_학사,대학기숙사A동(E)_학사,학사P/P(E)_학사,제2학생회관(E)_학사,교원아파트(E)_학사,대학C동(E)_학사,중앙연구기기센터(E)_학사,대학A동(E)_학사,다산빌딩(E)_학사,산학협력연구동(E)_학사
0,157.2,432.2,429.1,482.5,118.2,253.7,31.5,53.0,0.0,0.0,...,36.8,39.0,45.1,26.7,24.2,20.3,53.1,18.4,52.8,18.6
1,154.9,432.9,427.9,470.5,114.0,248.8,29.7,53.5,0.0,0.0,...,34.5,36.0,43.2,26.5,25.1,21.2,49.4,16.2,47.7,18.1
2,142.6,417.0,403.0,473.3,110.1,238.7,29.7,52.7,0.0,0.0,...,27.1,32.7,46.8,27.3,21.2,19.9,48.3,14.2,44.4,18.3
3,144.9,420.0,404.5,456.3,109.8,237.7,28.9,52.8,0.0,0.0,...,27.2,28.5,40.8,24.5,17.0,17.5,48.4,15.9,40.9,18.1
4,147.3,422.2,403.7,453.1,108.4,239.4,29.6,53.2,0.0,0.0,...,26.8,27.6,42.7,23.6,16.8,16.5,48.2,14.2,40.8,18.0
5,130.7,418.8,405.9,455.6,108.4,239.8,34.7,54.2,0.0,0.0,...,28.4,25.2,36.6,28.1,20.2,15.8,48.6,18.0,39.7,18.3
6,113.7,423.3,403.7,495.4,114.8,238.6,31.9,54.5,2.1,1.4,...,35.3,29.1,26.1,34.9,25.3,17.5,48.8,20.9,42.3,20.6
7,113.2,441.3,422.2,520.7,131.4,262.1,28.5,56.5,9.7,4.1,...,36.1,30.4,27.6,32.7,32.1,26.5,50.3,23.4,45.9,21.1
8,162.8,465.0,462.4,617.1,213.6,322.7,46.6,69.6,212.5,161.4,...,49.2,36.4,50.6,40.1,36.2,33.0,67.0,27.4,69.1,28.4
9,171.0,538.5,555.7,647.7,253.4,355.1,48.0,70.3,205.9,100.6,...,61.3,36.0,31.9,39.7,35.4,35.0,67.3,29.1,80.2,27.0


In [14]:
# * 실제 전기 부하량

same_day_y_test = energy_pred_model.y_test.loc[energy_pred_model.y_test.index.isin(same_day_data_indexes)]
same_day_y_test

,중앙P/P_석사,전기전자컴퓨터공학동_석사,신소재공학_석사,생명과학동_석사,기계공학동_석사,LG도서관_석사,창업B동_석사,금호관_석사,"냉각수순환펌프1,2,3_석사","냉각수순환펌프4,5,6_석사",...,대학B동(E)_학사,대학기숙사A동(E)_학사,학사P/P(E)_학사,제2학생회관(E)_학사,교원아파트(E)_학사,대학C동(E)_학사,중앙연구기기센터(E)_학사,대학A동(E)_학사,다산빌딩(E)_학사,산학협력연구동(E)_학사
120,154.4,447.5,421.0,520.7,139.0,267.6,37.6,50.7,0.0,0.0,...,37.9,39.3,36.2,33.7,25.6,29.4,45.9,14.9,52.6,17.2
121,155.0,413.8,418.3,456.0,138.4,271.0,27.7,50.2,0.0,0.0,...,38.6,39.3,44.1,24.7,26.6,14.7,46.0,27.6,49.9,17.6
122,142.1,411.5,404.9,446.9,123.4,269.3,27.1,50.4,0.0,0.0,...,38.6,34.6,44.5,20.9,21.8,15.4,46.0,27.6,44.5,16.9
123,149.7,405.6,410.4,447.1,120.3,269.0,27.3,50.8,0.0,0.0,...,25.3,28.3,36.0,23.9,14.5,14.0,45.7,14.8,34.1,18.7
124,144.6,407.8,400.7,505.8,126.8,256.0,27.0,49.9,0.0,0.0,...,23.4,33.6,35.4,22.9,13.5,14.2,45.7,13.8,33.7,17.9
125,112.0,404.8,402.8,459.6,123.6,255.1,28.5,50.6,0.0,0.0,...,24.9,22.0,18.6,28.4,25.5,13.6,46.8,14.3,32.6,16.6
126,109.0,411.8,413.2,477.0,118.0,265.2,34.7,55.8,0.0,0.0,...,34.7,30.0,20.3,32.7,34.6,16.3,47.0,26.1,38.3,19.6
127,103.1,437.9,429.4,510.5,145.2,273.8,31.6,61.9,210.7,139.0,...,29.2,27.6,37.0,32.8,27.7,30.6,47.4,19.3,45.9,20.1
128,135.2,449.6,464.5,578.8,209.5,323.0,55.9,72.1,209.7,138.5,...,54.8,34.3,57.7,44.8,35.6,31.7,61.4,31.8,67.0,22.7
129,148.6,518.7,522.5,681.4,311.7,375.7,43.6,78.1,209.9,138.1,...,49.7,46.0,25.2,60.7,41.6,30.9,67.4,32.5,78.4,21.2


In [15]:
import pandas as pd
# res_solar_sum = pd.DataFrame(res_solar.sum(axis=1), columns=['Total Predicted Solar'])
# res_solar_sum

res_load_sum = pd.DataFrame(res_load.sum(axis=1), columns=['Total Predicted Solar'])
res_load_sum

,Total Predicted Solar
0,6789.4
1,6707.1
2,6403.1
3,6322.8
4,6311.6
5,6325.5
6,6460.4
7,6929.2
8,8494.4
9,9428.1


In [16]:
# * 예측 라이브러리 추가
pip install pulp

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 14.3 MB 955 kB/s eta 0:00:01
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [38]:
# * 예측 발전량과 부하량을 통해 전기 요금 산출
from algorithm.schedule_algorithm import schedule

algorithm = schedule()

print(res_solar)
res_load_sum = res_load_sum["Total Predicted Solar"].values.tolist()
result = algorithm.schedule_optimizer([res_solar],[res_load_sum])


[6.474549553098411, 7.47409053309805, 3.9670980827240703, 3.9670980827240703, 3.9670980827240703, 6.899042526384995, 3.9670980827240703, 45.8567901332039, 202.2028532471457, 525.7670980993938, 656.3109500339858, 669.6513004528084, 648.6634674464402, 669.0924612857974, 720.2776647094879, 693.8709624738218, 577.1286605189911, 422.3947655514412, 210.30452073543165, 27.725144954920335, -3.2844280658840095, -3.619015134533825, -3.396024177128419, -3.396024177128419]
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/joyonghwan/Library/Python/3.9/lib/python/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/21/g_dwz6715z341ztvr9mvz6w80000gn/T/3484045f112c41da8f28261432046b9c-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/21/g_dwz6715z341ztvr9mvz6w80000gn/T/3484045f112c41da8f28261432046b9c-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 127 COLUMNS
At line 369 RHS
At line 492 BOU

In [40]:
# * 원으로 변경 및 자리수 맞추기
integer_result = int(result)

integer_result_= int(integer_result / 10) * 10
final_result = "{:,}".format(integer_result)
print(final_result+"원") 

24,609,148원
